In [1]:
import os

In [11]:
import cv2
import h5py
from ultralytics import YOLO
import os
from concurrent.futures import ThreadPoolExecutor
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'


def process_video_and_save_to_h5py(video_path, output_h5py_path, success_file_path, model_path='yolov8n.pt'):
    try:
        # Initialize YOLO model
        model = YOLO(model_path)

        # Open video file
        cap = cv2.VideoCapture(video_path)

        # Get total number of frames in the video
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        empty_frame_count = 0

        # Prepare to save tracking data
        with h5py.File(output_h5py_path, 'w') as h5file:
            dataset = h5file.create_group('tracking_data')

            frame_count = 0

            while cap.isOpened():
                # Read a frame from the video
                success, frame = cap.read()

                if not success:
                    break

                # Run YOLOv8 tracking on the frame using ByteTrack
                try:
                    results = model.track(frame, persist=True, tracker="bytetrack.yaml")
                except Exception as e:
                    print(f"Error in tracking on frame {frame_count} in {video_path}: {e}")
                    continue  # Skip to the next frame in case of an error

                # Proceed only if results are valid and contain tracking data
                if results and results[0].boxes.id is not None:
                    boxes = results[0].boxes.xywh.cpu().numpy()
                    track_ids = results[0].boxes.id.int().cpu().tolist()
                    confidences = results[0].boxes.conf.cpu().tolist()
                    class_ids = results[0].boxes.cls.int().cpu().tolist()

                    # Store features in h5py file
                    for i, (box, track_id, confidence, class_id) in enumerate(zip(boxes, track_ids, confidences, class_ids)):
                        track_data_group = dataset.create_group(f'frame_{frame_count:05}/track_{track_id}')
                        track_data_group.create_dataset('box', data=box)
                        track_data_group.create_dataset('confidence', data=confidence)
                        track_data_group.create_dataset('class_id', data=class_id)
                        track_data_group.create_dataset('track_id', data=track_id)
                else:
                    # Count this frame as empty if no results are returned
                    empty_frame_count += 1

                frame_count += 1

            # Release video capture object
            cap.release()

        # Log the successful processing
        with open(success_file_path, 'a') as f:
            f.write(f"{video_path}\n")

        return total_frames, empty_frame_count, frame_count, video_path

    except Exception as e:
        print(f"Error processing {video_path}: {e}")
        return None, None, None, video_path

def get_video_paths_from_folder(folder_path):
    return [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.mp4')]

def process_videos_in_parallel(video_paths, output_folder, success_file, failure_file, model_path='yolov5n.pt'):
    output_h5py_files = [os.path.join(output_folder, os.path.basename(video_path).replace('.mp4', '.h5')) for video_path in video_paths]

    successful_paths = []
    unsuccessful_paths = []

    with ThreadPoolExecutor(max_workers=350) as executor:
        futures = [executor.submit(process_video_and_save_to_h5py, video_path, output_h5py_path, success_file)
                   for video_path, output_h5py_path in zip(video_paths, output_h5py_files)]
        print("done")

        for future in futures:
            result = future.result()
            if result[0] is not None:
                successful_paths.append(result[3])
            else:
                unsuccessful_paths.append(result[3])

    with open(success_file, 'w') as f:
        for path in successful_paths:
            f.write(f"{path}\n")

    with open(failure_file, 'w') as f:
        for path in unsuccessful_paths:
            f.write(f"{path}\n")


In [12]:
#  __name__ == '__main__':
video_folder = r'Data\videos\anomaly-videos'  # Change to your folder path
anomalous_yolo_folder = r'Data\yolo-information\anomaly'
success_file = 'success.txt'
failure_file = 'failure.txt'
video_paths = get_video_paths_from_folder(video_folder)
if not os.path.exists(anomalous_yolo_folder):
    os.makedirs(anomalous_yolo_folder)
process_videos_in_parallel(video_paths, anomalous_yolo_folder, success_file, failure_file)

In [4]:
#  __name__ == '__main__':
video_folder = r'Data\sample_videos\normal-videos'  # Change to your folder path
normal_yolo_folder = r'Data\yolo-information\normal'
success_file = 'success.txt'
failure_file = 'failure.txt'
video_paths = get_video_paths_from_folder(video_folder)
if not os.path.exists(normal_yolo_folder):
    os.makedirs(normal_yolo_folder)
process_videos_in_parallel(video_paths, normal_yolo_folder, success_file, failure_file)



0: 480x640 7 persons, 228.4ms
Speed: 0.0ms preprocess, 228.4ms inference, 15.7ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 car, 232.8ms
Speed: 15.6ms preprocess, 232.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)


0: 480x640 1 car, 125.3ms
Speed: 0.0ms preprocess, 125.3ms inference, 25.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 7 persons, 126.0ms
Speed: 125.3ms preprocess, 126.0ms inference, 10.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 115.3ms
Speed: 49.4ms preprocess, 115.3ms inference, 19.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 7 persons, 116.3ms
Speed: 12.5ms preprocess, 116.3ms inference, 16.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 123.6ms
Speed: 9.0ms preprocess, 123.6ms inference, 11.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 7 persons, 125.4ms
Speed: 97.2ms preprocess, 125.4ms inference, 37.6ms postprocess per im

In [5]:
import h5py
import numpy as np
import os

def load_data_from_h5(folder_path):
    data = []
    labels = []
    
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.h5'):
            file_path = os.path.join(folder_path, file_name)
            with h5py.File(file_path, 'r') as h5file:
                for frame_key in h5file['tracking_data'].keys():
                    frame_data = h5file['tracking_data'][frame_key]
                    # Extract features for each frame
                    object_count = len(frame_data.keys())
                    class_distribution = np.zeros(80)  # Assuming 80 classes
                    confidences = []
                    
                    for track_id in frame_data.keys():
                        track_data = frame_data[track_id]
                        class_id = track_data['class_id'][()]
                        confidence = track_data['confidence'][()]
                        
                        class_distribution[class_id] += 1
                        confidences.append(confidence)
                    
                    # Normalize class distribution
                    if object_count > 0:
                        class_distribution /= object_count
                    
                    mean_confidence = np.mean(confidences) if confidences else 0
                    
                    # Create feature vector (you can add more features here)
                    features = np.concatenate((class_distribution, [mean_confidence, object_count]))
                    
                    data.append(features)
                    # Label based on folder name (0 for normal, 1 for anomalous)
                    label = 0 if 'normal' in folder_path else 1
                    labels.append(label)

    return np.array(data), np.array(labels)

In [6]:
from sklearn.model_selection import train_test_split

def preprocess_data(normal_folder, anomaly_folder):
    normal_data, normal_labels = load_data_from_h5(normal_folder)
    anomaly_data, anomaly_labels = load_data_from_h5(anomaly_folder)

    # Combine datasets
    data = np.concatenate((normal_data, anomaly_data), axis=0)
    labels = np.concatenate((normal_labels, anomaly_labels), axis=0)

    # Sample every fifth frame
    sampled_indices = np.arange(0, len(data), step=5)
    sampled_data = data[sampled_indices]
    sampled_labels = labels[sampled_indices]

    # Split into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(sampled_data, sampled_labels, test_size=0.2, random_state=42)

    return X_train, X_val, y_train, y_val

In [7]:
import torch
import torch.nn as nn

class AnomalyDetector(nn.Module):
    def __init__(self, input_size):
        super(AnomalyDetector, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=64, num_layers=2, batch_first=True)
        self.fc = nn.Linear(64, 1)  # Binary classification

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        output = self.fc(lstm_out[:, -1])  # Use the last time step's output
        return torch.sigmoid(output)  # Sigmoid for binary classification

In [8]:
def train_model(X_train, y_train):
    model = AnomalyDetector(input_size=X_train.shape[1])
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    model.train()
    
    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train).float().unsqueeze(1)  # Add sequence dimension
    y_train_tensor = torch.tensor(y_train).float().unsqueeze(1)

    for epoch in range(50):  # Number of epochs can be adjusted
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/50], Loss: {loss.item():.4f}')

    return model

In [9]:
from sklearn.metrics import roc_auc_score

def evaluate_model(model, X_val, y_val):
    model.eval()
    
    with torch.no_grad():
        # Convert validation data to tensor and add sequence dimension
        X_val_tensor = torch.tensor(X_val).float().unsqueeze(1)  # Add sequence dimension
        
        # Get model outputs
        outputs = model(X_val_tensor)
        
        # Convert outputs to numpy and flatten
        predicted_probs = outputs.numpy().flatten()  # Get probabilities for the positive class
        
        # Convert labels to numpy
        y_val_np = y_val.flatten()  # Ensure y_val is flat for comparison

        # Calculate predicted labels based on threshold
        predicted_labels = (predicted_probs > 0.5).astype(int)

        # Calculate accuracy
        accuracy = np.mean(predicted_labels == y_val_np)
        
        # Calculate ROC AUC
        roc_auc = roc_auc_score(y_val_np, predicted_probs)

        print(f'Validation Accuracy: {accuracy:.4f}')
        print(f'ROC AUC Score: {roc_auc:.4f}')

In [10]:
X_train, X_val, y_train, y_val = preprocess_data(normal_yolo_folder, anomalous_yolo_folder)
model = train_model(X_train, y_train)

evaluate_model(model, X_val, y_val)


Epoch [10/50], Loss: 0.6688
Epoch [20/50], Loss: 0.6479
Epoch [30/50], Loss: 0.6200
Epoch [40/50], Loss: 0.5928
Epoch [50/50], Loss: 0.5737
Validation Accuracy: 0.6875
ROC AUC Score: 0.6227


In [37]:
X_train[0]

array([          0,           0,         0.5,           0,           0,           0,           0,         0.5,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,
                 0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,
                 0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,
                 0,           0,           

In [40]:
load_data_from_h5(anomaly_folder)

(array([[        0.5,           0,         0.5, ...,           0,     0.71955,           2],
        [        0.5,           0,         0.5, ...,           0,     0.71325,           2],
        [        0.5,           0,         0.5, ...,           0,     0.52827,           2],
        ...,
        [          1,           0,           0, ...,           0,     0.56352,           3],
        [          1,           0,           0, ...,           0,     0.59748,           4],
        [       0.75,           0,           0, ...,           0,     0.46724,           4]]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 